In [1]:
import pandas as pd
import mwclient

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
leagues = get_leagues(where='L.League_Short in ("LCK", "LPL", "LEC", "LCS", "WCS", "MSI")')
leagues

,League,League Short,Region,Level,IsOfficial
0,League of Legends Championship Series,LCS,North America,Primary,Yes
1,LoL Champions Korea,LCK,Korea,Primary,Yes
2,LoL European Championship,LEC,Europe,Primary,Yes
3,Mid-Season Invitational,MSI,International,Primary,Yes
4,Tencent LoL Pro League,LPL,China,Primary,Yes
5,World Championship,WCS,International,Primary,Yes


In [3]:
tournaments = None
for league in leagues['League Short']:
    tournament = get_tournaments(where=f'L.League_Short="{league}"')
    if tournaments is not None:
        tournaments = pd.concat([tournaments, tournament])
    else:
        tournaments = tournament
    
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,LCS 2019 Spring,LCS/2019 Season/Spring Season,2019-01-26,2019-03-24,League of Legends Championship Series,North America,Offline,LCS 2019 Spring,Spring,1,Primary,0,0,1,2019,1,1
1,LCS 2019 Spring Playoffs,LCS/2019 Season/Spring Playoffs,2019-03-30,2019-04-13,League of Legends Championship Series,North America,Offline,LCS 2019 Spring Playoffs,Spring,None,Primary,0,1,1,2019,1,1
2,LCS 2019 Summer,LCS/2019 Season/Summer Season,2019-06-01,2019-08-04,League of Legends Championship Series,North America,Offline,LCS 2019 Summer,Summer,2,Primary,0,0,1,2019,1,1
3,LCS 2019 Summer Playoffs,LCS/2019 Season/Summer Playoffs,2019-08-10,2019-08-25,League of Legends Championship Series,North America,Offline,LCS 2019 Summer Playoffs,Summer,None,Primary,0,1,1,2019,1,1
4,LCS 2020 Spring,LCS/2020 Season/Spring Season,2020-01-25,2020-03-29,League of Legends Championship Series,North America,Offline/Online,LCS 2020 Spring,Spring,1,Primary,0,0,1,2020,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Worlds 2022 Main Event,2022 Season World Championship/Main Event,2022-10-07,2022-11-05,World Championship,International,Offline,Worlds 2022 Main Event,None,None,Primary,0,0,1,2022,1,1
63,Worlds 2022 Play-In,2022 Season World Championship/Play-In,2022-09-29,2022-10-04,World Championship,International,Offline,Worlds 2022 Play-In,None,None,Primary,1,0,1,2022,1,1
64,Worlds Season 1,Season 1 World Championship,2011-06-18,2011-06-20,World Championship,International,Offline,Worlds Season 1,None,None,Primary,0,0,1,2011,1,1
65,Worlds Season 2,Season 2 World Championship,2012-10-04,2012-10-13,World Championship,International,Offline,Worlds Season 2,None,None,Primary,0,0,1,2012,1,1


In [4]:
scoreboard_games = None
for overview in tournaments['OverviewPage']:
    try:
        scoreboard_game = get_scoreboard_games(where=f'T.OverviewPage="{overview}"')
        if scoreboard_games is not None:
            scoreboard_games = pd.concat([scoreboard_games, scoreboard_game])
        else:
            scoreboard_games = scoreboard_game
    except Exception as e:
        scoreboard_game = get_scoreboard_games(where=f'T.OverviewPage="{overview}"', casting=False)
        print(e)
        print(scoreboard_game)
        break

scoreboard_games = scoreboard_games.reset_index(drop=True)
scoreboard_games

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision
0,LCS/2019 Season/Spring Season,100 Thieves,Clutch Gaming,100 Thieves,Clutch Gaming,2019-02-09 21:34:00,1,0,1,51:21,51.350000,"Rumble,Lee Sin,Sylas,Tahm Kench,Gragas","Cassiopeia,Lucian,Akali,Sivir,Kai'Sa","Urgot,Aatrox,Lissandra,Vayne,Alistar","Zilean,Jayce,Zoe,Ezreal,Braum","Ssumday,AnDa,huhi,Bang,aphromoo","Huni,Lira,Damonte,Piglet,Vulcan (Philippe Lafl...",4.0,2.0,2.0,1.0,11.0,2.0,93817.0,85945.0,19.0,7.0,1.0,0.0,4.0,0.0,9.2,LCS/2019 Season/Spring Season_Week 3_1_1,LCS/2019 Season/Spring Season_Week 3_1,1053197,0
1,LCS/2019 Season/Spring Season,100 Thieves,Counter Logic Gaming,100 Thieves,Counter Logic Gaming,2019-02-10 23:52:00,1,0,1,41:07,41.116667,"Sion,Tahm Kench,Zilean,Syndra,Orianna","Urgot,Akali,Cassiopeia,Zoe,Lissandra","Aatrox,Camille,Ryze,Ezreal,Braum","Ornn,Xin Zhao,Viktor,Lucian,Thresh","Ssumday,AnDa,huhi,Bang,aphromoo","Darshan,Wiggily,PowerOfEvil,Stixxay,Biofrost",5.0,1.0,2.0,0.0,11.0,1.0,83579.0,67401.0,12.0,8.0,1.0,0.0,4.0,0.0,9.2,LCS/2019 Season/Spring Season_Week 3_10_1,LCS/2019 Season/Spring Season_Week 3_10,1053459,0
2,LCS/2019 Season/Spring Season,100 Thieves,Counter Logic Gaming,Counter Logic Gaming,100 Thieves,2019-03-24 00:23:00,0,1,2,33:47,33.783333,"Rek'Sai,Nocturne,LeBlanc,Gangplank,Kennen","Lissandra,Sylas,Jayce,Irelia,Skarner","Akali,Jarvan IV,Galio,Xayah,Rakan","Poppy,Sejuani,Syndra,Kalista,Braum","FakeGod,Fragas,Soligo,Bang,Stunt","Darshan,Wiggily,PowerOfEvil,Stixxay,Biofrost",3.0,2.0,0.0,2.0,3.0,9.0,50917.0,61896.0,5.0,14.0,0.0,1.0,0.0,2.0,9.5,LCS/2019 Season/Spring Season_Week 9_5_1,LCS/2019 Season/Spring Season_Week 9_5,1074215,0
3,LCS/2019 Season/Spring Season,100 Thieves,Echo Fox,Echo Fox,100 Thieves,2019-02-17 19:46:00,0,1,2,29:06,29.100000,"Thresh,Nocturne,Ezreal,Sion,Lee Sin","Lucian,Urgot,Cassiopeia,Aurelion Sol,Sejuani","Jayce,Olaf,Lissandra,Ashe,Tahm Kench","Malphite,Jarvan IV,Zoe,Sivir,Alistar","Ssumday,AnDa,huhi,Bang,aphromoo","Solo (Colin Earnest),Rush,Fenix,Apollo (Apollo...",0.0,4.0,0.0,1.0,2.0,9.0,44174.0,54437.0,4.0,14.0,0.0,1.0,0.0,2.0,9.3,LCS/2019 Season/Spring Season_Week 4_6_1,LCS/2019 Season/Spring Season_Week 4_6,1012442,0
4,LCS/2019 Season/Spring Season,100 Thieves,FlyQuest,FlyQuest,100 Thieves,2019-03-17 00:52:00,0,1,2,34:53,34.883333,"Riven,Lissandra,Thresh,Nocturne,Ryze","Jayce,Lucian,Neeko,Elise,Skarner","Vladimir,Jarvan IV,Zoe,Ezreal,Braum","Aatrox,Rek'Sai,LeBlanc,Xayah,Rakan","Ssumday,AnDa,Soligo,Bang,aphromoo","V1per,Santorin,Pobelter,WildTurtle,JayJ",0.0,4.0,0.0,1.0,0.0,11.0,57387.0,66432.0,4.0,8.0,0.0,1.0,0.0,3.0,9.5,LCS/2019 Season/Spring Season_Week 8_5_1,LCS/2019 Season/Spring Season_Week 8_5,1071794,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13183,Season 3 World Championship,Team Vulcun,Samsung Ozone,Samsung Ozone,Team Vulcun,2013-09-21 20:00:00,0,1,2,36:33,36.550000,"Caitlyn,Lee Sin,Twisted Fate","Ahri,Corki,Zed","Shen,Jarvan IV,Twisted Fate,Ashe,Zyra","Singed,Elise,Ezreal,Vayne,Sona","Sycho Sid,Xmithie,mancloud,Zuna,BloodWater","Looper,DanDy,Dade,imp,Mata",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Season 3 World Championship_Day 6_2_1,Season 3 World Championship_Day 6_2,None,0
13184,Season 3 World Championship,TSM,GamingGear.eu,GamingGear.eu,TSM,2013-09-22 03:00:00,0,1,2,34:24,34.400000,"Aatrox,Elise,Zyra","Shen,Vi,Zed","Vladimir,Hecarim,Orianna,Teemo,Thresh","Tryndamere,Jarvan IV,Gragas,Corki,Sona","Xpecial,TheOddOne,WildTurtle,Reginald,Dyrus","Nbs,Alunir,Mazzerin,DeadlyBrother,Inspirro",NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [64]:
scoreboard_games['Team1GPM'] = scoreboard_games['Team1Gold'] / scoreboard_games['Gamelength Number']
scoreboard_games['Team2GPM'] = scoreboard_games['Team2Gold'] / scoreboard_games['Gamelength Number']
scoreboard_games['Team1KPM'] = scoreboard_games['Team1Kills'] / scoreboard_games['Gamelength Number']
scoreboard_games['Team2KPM'] = scoreboard_games['Team2Kills'] / scoreboard_games['Gamelength Number']

In [66]:
def index_name(df):
    df.index = df.index.set_names('Team', level=1)
    return df

teams = pd.DataFrame()
grouped1 = scoreboard_games.groupby(['OverviewPage', 'Team1'])
grouped2 = scoreboard_games.groupby(['OverviewPage', 'Team2'])
games1 = index_name(grouped1['Team2'].count())
games2 = index_name(grouped2['Team1'].count())
teams['Games'] = games1 + games2
result1 = index_name(grouped1['Winner'].value_counts().unstack(fill_value=0))
result2 = index_name(grouped2['Winner'].value_counts().unstack(fill_value=0))
teams['Win'] = result1[1] + result2[2]
teams['Win'] = teams['Win'].fillna(0).astype('int')
teams['Loss'] = result1[2] + result2[1]
teams['Loss'] = teams['Loss'].fillna(0).astype('int')
teams['WinRate'] = teams['Win'] / teams['Games']
length1 = index_name(grouped1['Gamelength Number'].sum())
length2 = index_name(grouped2['Gamelength Number'].sum())
teams['Gamelength Number'] = length1 + length2
teams['Gamelength Number'] = teams['Gamelength Number'] / teams['Games']
gpm1 = index_name(grouped1['Team1GPM'].sum())
gpm2 = index_name(grouped2['Team2GPM'].sum())
teams['GPM'] = gpm1 + gpm2
teams['GPM'] = teams['GPM'] / teams['Games']
kpm1 = index_name(grouped1['Team1KPM'].sum())
kpm2 = index_name(grouped2['Team2KPM'].sum())
teams['KPM'] = kpm1 + kpm2
teams['KPM'] = teams['KPM'] / teams['Games']
teams

Games  Win  Loss  \
OverviewPage                       Team                                      
2014 Season Garena Regional Finals Saigon Fantastic Five   12.0    6     6   
                                   Saigon Jokers            4.0    1     3   
                                   Taipei Snipers           5.0    2     3   
                                   ahq eSports Club         3.0    3     0   
2014 Season Korea Regional Finals  KT Rolster Arrows        3.0    0     3   
...                                                         ...  ...   ...   
Season 3 World Championship        Royal Club               9.0    5     4   
                                   SK Telecom T1           18.0   15     3   
                                   Samsung Ozone            9.0    5     4   
                                   TSM                      8.0    2     6   
                                   Team Vulcun              8.0    3     5   

                                                           WinRate  \
OverviewPage                       Team                              
2014 Season Garena Regional Finals Saigon Fantastic Five  0.500000   
                                   Saigon Jokers          0.250000   
                                   Taipei Snipers         0.400000   
                                   ahq eSports Club       1.000000   
2014 Season Korea Regional Finals  KT Rolster Arrows      0.000000   
...                                                            ...   
Season 3 World Championship        Royal Club             0.555556   
                                   SK Telecom T1          0.833333   
                                   Samsung Ozone          0.555556   
                                   TSM                    0.250000   
                                   Team Vulcun            0.375000   

                                                          Gamelength Number  \
OverviewPage                       Team                                       
2014 Season Garena Regional Finals Saigon Fantastic Five          32.340278   
                                   Saigon Jokers                  28.829167   
                                   Taipei Snipers                 34.113333   
                                   ahq eSports Club               34.066667   
2014 Season Korea Regional Finals  KT Rolster Arrows              40.638889   
...                                                                     ...   
Season 3 World Championship        Royal Club                     32.066667   
                                   SK Telecom T1                  31.558333   
                                   Samsung Ozone                  31.801852   
                                   TSM                            35.831250   
                                   Team Vulcun                    30.447917   

                                                          GPM  KPM  
OverviewPage                       Team                             
2014 Season Garena Regional Finals Saigon Fantastic Five  0.0  0.0  
                                   Saigon Jokers          0.0  0.0  
                                   Taipei Snipers         0.0  0.0  
                                   ahq eSports Club       0.0  0.0  
2014 Season Korea Regional Finals  KT Rolster Arrows      0.0  0.0  
...                                                       ...  ...  
Season 3 World Championship        Royal Club             0.0  0.0  
                                   SK Telecom T1          0.0  0.0  
                                   Samsung Ozone          0.0  0.0  
                                   TSM                    0.0  0.0  
                                   Team Vulcun            0.0  0.0  

[1456 rows x 7 columns]

In [61]:
"""
k:d, GPM, GDM, Kills/game
Deaths / game
towers killed
towers lost
fb%
ft%
csm
dpm
wpm
vwpm
wcpm

"""
#  & (scoreboard_games['Team1'] == 'Flash Wolves' | scoreboard_games['Team2'] == 'Flash Wolves')

0        93817.0
1        83579.0
2        50917.0
3        44174.0
4        57387.0
          ...   
13183        NaN
13184        NaN
13185        NaN
13186        NaN
13187        NaN
Name: Team1Gold, Length: 13188, dtype: float64